<a href="https://colab.research.google.com/github/MarciaFG/scimobility/blob/main/LaplacianMatrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 800*800 Laplacian Matrix from Meso-Fields

* Calculate the Laplacian matrix of the square matrix.
* Calculate the eigenvalues and eigenvectors of the Laplacian matrix.
* Sort the eigenvalues in ascending order.
* The second smallest eigenvalue corresponds to the second eigenvector.
* Normalize the second eigenvector to have a unit length.

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.
Your runtime has 13.6 gigabytes of available RAM

To enable a high-RAM runtime, select the Runtime → "Change runtime type"
menu, and then select High-RAM in the Runtime shape dropdown. Then, 
re-execute this cell.


In [2]:
import numpy as np
import requests
import pandas as pd
from tqdm import tqdm
import torch
import nltk
import io
import matplotlib.pyplot as plt
from google.cloud import bigquery
from google.colab import files
import humanize
#!pip install google-sheets-to-csv
# enable data table format
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [3]:
import os.path
from google.colab import drive

# mount Google Drive to /content/drive/My Drive/
if os.path.isdir("/content/drive/My Drive"):
  print("Google Drive already mounted")
else:
  drive.mount('/content/drive/')

Mounted at /content/drive/


In [11]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
id = '17RXABV5bp0-u_oSvKmKONW3f7FLzrjYl'
print (id) # Verify that you have everything
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('MesoFieldsMatrix.csv')  
Matrix = pd.read_csv('MesoFieldsMatrix.csv', sep=";") # Dataset is now stored in a Pandas Dataframe
Matrix.head(1)

17RXABV5bp0-u_oSvKmKONW3f7FLzrjYl


,source,target,weight
0,0,1,5417


In [ ]:
import numpy as np
from scipy.sparse import coo_matrix, diags

# Create an example edge list
edgelist = np.array([
    [0, 1, 1.0],
    [0, 2, 1.0],
    [1, 2, 0.5],
    [2, 3, 1.0],
    [3, 4, 1.0],
    [3, 5, 1.0],
    [4, 5, 1.0]
])

# Get the number of nodes in the graph
num_nodes = np.max(edgelist[:, :2]) + 1

# Create a sparse matrix in COO format
data = edgelist[:, 2]
row = edgelist[:, 0]
col = edgelist[:, 1]
adj_matrix = coo_matrix((data, (row, col)), shape=(num_nodes, num_nodes))

# Compute the degree matrix
degree_matrix = diags(adj_matrix.sum(axis=1).flatten(), 0)

# Compute the Laplacian matrix
laplacian_matrix = degree_matrix - adj_matrix

# Print the Laplacian matrix
print(laplacian_matrix.toarray())